In [1]:
import tensorflow as tf
import catboost

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, recall_score, plot_precision_recall_curve, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd     
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from imblearn.over_sampling import RandomOverSampler

pio.renderers.default='notebook'
import xgboost as xgb




In [2]:
def normalize(df, how="min-max"):
    if how == "min-max" :
        return (df - df.min())/(df.max()-df.min())
    if how == "mean" :
        return (df-df.mean())/df.std()




def get_cp_indices_df(df_cp, df_indices, selected_coutry = None, data_resolution = "M", normalization="mean", normalize_cp=True, smoothing = False, smoothing_duration=6, indices=None) :


    if not selected_coutry : selected_coutry = df_cp.country.unique()

    df_cp_filter = df_cp[df_cp.country.isin(selected_coutry) ]
    df_cp_filter = df_cp_filter.groupby([df_cp_filter.index.to_period(data_resolution)]).mean().reset_index()
    df_cp_filter = df_cp_filter.set_index(df_cp_filter.timestamp).drop(["timestamp"], axis=1)


    if indices : df_indices = df_indices[indices]
    df_indices_filter = df_indices.groupby([df_indices.index.to_period(data_resolution)]).mean().reset_index()
    df_indices_filter = df_indices_filter.set_index(df_indices_filter.timestamp).drop(["timestamp"], axis=1)

    df_cp_indices = df_cp_filter.join(df_indices_filter)
    df_cp_indices = df_cp_indices[df_cp_indices.index.year>=1982]
    df_cp_indices = normalize(df_cp_indices,how=normalization)


    if not normalize_cp :  
        df_cp_indices.capacity_factor = df_cp_filter[df_cp_filter.index.year>=1982]


    if smoothing : 
        df_cp_indices = df_cp_indices.rolling(smoothing_duration, center=True).mean()


    return df_cp_indices

    

def compare_cp_index(df_cp_indices, smoothing = False, smoothing_duration=6, indices = None, start=1990, end=2019, show_plot = True):
    if indices : 
        df_cp_indices = df_cp_indices[indices+["capacity_factor"]] 
    df_cp_indices = df_cp_indices[(df_cp_indices.index.year > start) & (df_cp_indices.index.year < end)]
    
    if show_plot : 
        fig = px.line(df_cp_indices.set_index(df_cp_indices.index.astype("str")), 
            title= ("normalized capacity factor and climate indices, smoothing={}".format(smoothing_duration) if smoothing else "normalized capacity factor and climate indices"))
        fig.update_traces(visible='legendonly')
        fig.update_traces(visible=True, selector=dict(name="capacity_factor"))
        fig.show()
        
    
    return df_cp_indices.corr().capacity_factor[:-1]


def timeseries_generator(dataset,window_size,target="capacity_factor") :
    X = dataset.drop(target, axis = 1 )
    new_y = dataset[target].iloc[window_size:]
    new_X = np.array([X[i:i+window_size] for i in range(X.shape[0]-window_size)],dtype=object)
    return new_X, new_y

 

In [3]:
df_cp = pd.read_csv("dataset_with_timestamp")
df_cp = df_cp.drop(['hour', 'month', 'year', 'day'], axis=1)
df_cp['timestamp'] = pd.to_datetime(df_cp['timestamp'], format='%Y-%m-%d %H:%M:%S')

df_cp = df_cp.groupby(["timestamp","country"]).mean().reset_index()
df_cp = df_cp.set_axis(df_cp.timestamp)
df_cp = df_cp.drop(["timestamp"], axis=1)

indices = ["nao", "ao","mjo80e","mjo40w","mjo20e","mjo160e","mjo10w","nino34"]

df_indices = pd.read_csv("daily_indices_82_to_19.csv")
df_indices['timestamp'] = pd.to_datetime(df_indices['timestamp'], format='%Y-%m-%d')
df_indices = df_indices[["timestamp"]+indices]
df_indices = df_indices.set_axis(df_indices.timestamp)
df_indices = df_indices.drop(["timestamp"], axis=1)




compute correlation between LWP and CF of countries :

In [4]:
cp = []
for country in df_cp.country.unique() :
    cp.append(df_cp[df_cp.country==country].capacity_factor.to_list())

cp_countries = pd.DataFrame(np.array(cp).T, columns = df_cp.country.unique())
LWP_countries = pd.DataFrame((np.array(cp).T <=0.1), columns = df_cp.country.unique())

In [21]:

melted = LWP_countries.corr().melt(ignore_index=False, var_name="main_country")
fig=px.choropleth_mapbox(melted,
        geojson="https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson",
        featureidkey='properties.ISO2',   
        locations=melted.index,        #column in dataframe
        animation_frame =melted.main_country,       #dataframe
        color=melted.value,  #dataframe
            zoom=1, center = {"lat": 56.4, "lon": 15.0},
            range_color=[0,0.6],
            mapbox_style="carto-positron",
            color_continuous_scale="Viridis",
            opacity = 0.5,
        title='Correlation between smoothened monthly capacity factor and different climate indices' ,  
        )
fig.show()

# With capacity factor of other countries :

## All countries

In [ ]:
data_resolution = "d"
normalize_cp=True

indices = ["nao","ao"]
main_country = ["DE"]

df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
df_cp_indices["LWP"] = (df_cp_indices_country.capacity_factor<0.1).astype("int")
countries = df_cp.country.unique()

for window_size in [1,10,50,100,200] :
    print("window_size",window_size)



    Xs = []
    ys = []

    for country in countries :
        df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
        df_cp_indices[country] = df_cp_indices_country.capacity_factor



    X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")


    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

    X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
    X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


    oversample = RandomOverSampler(sampling_strategy='minority')
    X_train, y_train = oversample.fit_resample(X_train, y_train)


    clf = catboost.CatBoostClassifier()
    clf.fit(X_train,y_train, verbose=False)
    y_pred = clf.predict(X_test)


    df_test = pd.DataFrame(y_test, columns =["LWP"])
    df_test["LWP_pred"] = y_pred
    

    print("""Recall :
    What proportion of actual positives was identified correctly""",recall_score(y_test,y_pred))
    print("""Precision:
    What proportion of positive identifications was actually correctly""",precision_score(y_test,y_pred))
    print("""Accuracy:
    """,accuracy_score(y_test,y_pred))




## Using neigbors

In [ ]:
data_resolution = "d"
normalize_cp=True

indices = ["nao","ao"]
main_country = ["DE"]

df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
df_cp_indices["LWP"] = (df_cp_indices_country.capacity_factor<0.1).astype("int")
countries = LWP_countries.corr()["DE"].sort_values(ascending=False).index[1:11]

for window_size in [20] :
    print("window_size",window_size)



    Xs = []
    ys = []

    for country in countries :
        if country not in main_country:
            df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
            df_cp_indices[country] = df_cp_indices_country.capacity_factor



    X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")


    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

    X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
    X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


    oversample = RandomOverSampler(sampling_strategy='minority')
    X_train, y_train = oversample.fit_resample(X_train, y_train)


    clf = catboost.CatBoostClassifier()
    clf.fit(X_train,y_train, verbose=True)
    y_pred = clf.predict(X_test)


    df_test = pd.DataFrame(y_test, columns =["LWP"])
    df_test["LWP_pred"] = y_pred
    

    print("""Recall :
    What proportion of actual positives was identified correctly""",recall_score(y_test,y_pred))
    print("""Precision:
    What proportion of positive identifications was actually correctly""",precision_score(y_test,y_pred))
    print("""Accuracy:
    """,accuracy_score(y_test,y_pred))




# With LWP of other countries :

## All countries

In [ ]:
data_resolution = "d"
normalize_cp=True

indices = ["nao","ao"]
main_country = ["DE"]

df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
df_cp_indices["LWP"] = (df_cp_indices_country.capacity_factor<0.1).astype("int")
countries = df_cp.country.unique()

for window_size in [50,100,200] :
    print("window_size",window_size)



    Xs = []
    ys = []

    for country in countries :
        if country not in main_country:
            df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
            df_cp_indices[country] = (df_cp_indices_country.capacity_factor<0.1)



    X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")


    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

    X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
    X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


    oversample = RandomOverSampler(sampling_strategy='minority')
    X_train, y_train = oversample.fit_resample(X_train, y_train)


    clf = catboost.CatBoostClassifier()
    clf.fit(X_train,y_train, verbose=True)
    y_pred = clf.predict(X_test)


    df_test = pd.DataFrame(y_test, columns =["LWP"])
    df_test["LWP_pred"] = y_pred
    

    print("""Recall :
    What proportion of actual positives was identified correctly""",recall_score(y_test,y_pred))
    print("""Precision:
    What proportion of positive identifications was actually correctly""",precision_score(y_test,y_pred))
    print("""Accuracy:
    """,accuracy_score(y_test,y_pred))




## Using Neigbors and CP of main country

In [31]:
data_resolution = "d"
normalize_cp=True
indices = ["nao","ao"]
for main_country in  ["FR","DE","GB","DK","NO"] :

    print("country: ", main_country)

    df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[main_country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
    df_cp_indices["LWP"] = (df_cp_indices.capacity_factor<0.1).astype("int")
    countries = LWP_countries.corr()[main_country].sort_values(ascending=False).index[1:11]




    Xs = []
    ys = []

    for country in countries :
        if country not in main_country:
            df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
            df_cp_indices[country] = (df_cp_indices_country.capacity_factor<0.1)

    for window_size in [1,10,20,50,200] :
        print("""   window_size =""", window_size)

        X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")


        

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

        X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
        X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


        oversample = RandomOverSampler(sampling_strategy='minority')
        X_train, y_train = oversample.fit_resample(X_train, y_train)


        clf = catboost.CatBoostClassifier()
        clf.fit(X_train,y_train, verbose=False)
        y_pred = clf.predict(X_test)


        df_test = pd.DataFrame(y_test, columns =["LWP"])
        df_test["LWP_pred"] = y_pred

        y_pred_proba = clf.predict_proba(X_test)[:, 1] 

        print("""       Recall:""",recall_score(y_test,y_pred))
        print("""       Precision:""",precision_score(y_test,y_pred))
        print("""       Accuracy:""",accuracy_score(y_test,y_pred))
        print("""       Roc auc:""",roc_auc_score(y_test.values, y_pred_proba))





country:  FR
   window_size = 1
       Recall: 0.6463932107496464
       Precision: 0.35619641465315666
       Accuracy: 0.7650655021834061
       Roc auc: 0.8049255517562378
   window_size = 10
       Recall: 0.5155367231638418
       Precision: 0.4269005847953216
       Accuracy: 0.8180030587721214
       Roc auc: 0.8206134375369627
   window_size = 20
       Recall: 0.4862914862914863
       Precision: 0.43149807938540335
       Accuracy: 0.825098382160035
       Roc auc: 0.8185097561546929
   window_size = 50
       Recall: 0.4692982456140351
       Precision: 0.4579172610556348
       Accuracy: 0.8372042068361086
       Roc auc: 0.8259582052209562
   window_size = 200
       Recall: 0.46107784431137727
       Precision: 0.43441466854724964
       Accuracy: 0.8314507198228128
       Roc auc: 0.8130633715672373
country:  DE
   window_size = 1
       Recall: 0.7193211488250653
       Precision: 0.3932905067808708
       Accuracy: 0.767467248908297
       Roc auc: 0.8268455093985194
 

# increasing threshold 

In [72]:
threshold = 0.2

In [73]:
cp = []
for country in df_cp.country.unique() :
    cp.append(df_cp[df_cp.country==country].capacity_factor.to_list())

cp_countries = pd.DataFrame(np.array(cp).T, columns = df_cp.country.unique())
LWP_countries = pd.DataFrame((np.array(cp).T <=threshold), columns = df_cp.country.unique())

In [75]:

melted = LWP_countries.corr().melt(ignore_index=False, var_name="main_country")
fig=px.choropleth_mapbox(melted,
        geojson="https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson",
        featureidkey='properties.ISO2',   
        locations=melted.index,        #column in dataframe
        animation_frame =melted.main_country,       #dataframe
        color=melted.value,  #dataframe
            zoom=1, center = {"lat": 56.4, "lon": 15.0},
            range_color=[0,0.6],
            mapbox_style="carto-positron",
            color_continuous_scale="Viridis",
            opacity = 0.5,
        title='Correlation between LWP(0.2) of main country and LWP(0.2) of other countries ' ,  
        )
fig.show()

## Using LWP neighbors and CP of main coutry

In [6]:
data_resolution = "d"
normalize_cp=True
indices = ["nao","ao"]
for main_country in  ["FR","DE","GB","DK","NO"] :

    print("country: ", main_country)

    df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[main_country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
    df_cp_indices["LWP"] = (df_cp_indices.capacity_factor<0.2).astype("int")
    countries = LWP_countries.corr()[main_country].sort_values(ascending=False).index[1:11]




    Xs = []
    ys = []

    for country in countries :
        if country not in main_country:
            df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
            df_cp_indices[country] = (df_cp_indices_country.capacity_factor<0.2)

    for window_size in [1,10,20,50,200] :
        print("""   window_size =""", window_size)

        X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")


        

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

        X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
        X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


        oversample = RandomOverSampler(sampling_strategy='minority')
        X_train, y_train = oversample.fit_resample(X_train, y_train)


        clf = catboost.CatBoostClassifier()
        clf.fit(X_train,y_train, verbose=False)
        y_pred = clf.predict(X_test)


        df_test = pd.DataFrame(y_test, columns =["LWP"])
        df_test["LWP_pred"] = y_pred

        y_pred_proba = clf.predict_proba(X_test)[:, 1] 

        print("""       Recall:""",recall_score(y_test,y_pred))
        print("""       Precision:""",precision_score(y_test,y_pred))
        print("""       Accuracy:""",accuracy_score(y_test,y_pred))
        print("""       Roc auc:""",roc_auc_score(y_test.values, y_pred_proba))





country:  FR
   window_size = 1
       Recall: 0.7583756345177665
       Precision: 0.6616474756421612
       Accuracy: 0.7292576419213974
       Roc auc: 0.8109236244821751
   window_size = 10
       Recall: 0.7252412392077197
       Precision: 0.675177304964539
       Accuracy: 0.7317019882018789
       Roc auc: 0.8183230019286674
   window_size = 20
       Recall: 0.7461695607763024
       Precision: 0.6811188811188811
       Accuracy: 0.7418014866637517
       Roc auc: 0.8176420034798038
   window_size = 50
       Recall: 0.7162718846549948
       Precision: 0.6742607852641784
       Accuracy: 0.7320333041191937
       Roc auc: 0.8147220971876248
   window_size = 200
       Recall: 0.7152284263959391
       Precision: 0.6968348170128585
       Accuracy: 0.7399778516057586
       Roc auc: 0.8206023555692957
country:  DE
   window_size = 1
       Recall: 0.751937984496124
       Precision: 0.6566731141199227
       Accuracy: 0.7471615720524017
       Roc auc: 0.8215968401411301
   wi

KeyboardInterrupt: 

## Without CP of main country

In [91]:
data_resolution = "d"
normalize_cp=True
indices = ["nao"]


df_results = pd.DataFrame(columns=["country","window_size","recall", "precision", "accuracy", "ROC_AUC"])

for main_country in df_cp.country.unique():

    print("country: ", main_country)

    df_cp_indices = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[main_country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
    df_cp_indices["LWP"] = (df_cp_indices.capacity_factor<0.2).astype("int")
    df_cp_indices.drop(["capacity_factor","nao"], axis=1, inplace=True)
    countries = LWP_countries.corr()[main_country].sort_values(ascending=False).index[1:11]




    Xs = []
    ys = []

    for country in countries :
        if country not in main_country:
            df_cp_indices_country = get_cp_indices_df(df_cp, df_indices,indices=indices, selected_coutry=[country], data_resolution=data_resolution, normalization="mean",normalize_cp=False, smoothing=False, smoothing_duration=1).dropna()
            df_cp_indices[country] = (df_cp_indices_country.capacity_factor<0.2).astype("int")

    for window_size in [1,10,20] :
        print("""   window_size =""", window_size)

        X, y = timeseries_generator(df_cp_indices,window_size,target="LWP")

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle = True)

        X_train=X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
        X_test=X_test.reshape(-1,X_test.shape[1]*X_test.shape[2])


        oversample = RandomOverSampler(sampling_strategy='minority')
        X_train, y_train = oversample.fit_resample(X_train, y_train)



        pool_train = catboost.Pool(X_train, y_train,
                        cat_features = list(range(X_test[0].shape[0])))

        pool_test = catboost.Pool(X_test, cat_features = list(range(X_test[0].shape[0])))



        clf = catboost.CatBoostClassifier()
        clf.fit(pool_train, verbose=False)
        y_pred = clf.predict(pool_test)


        df_test = pd.DataFrame(y_test, columns =["LWP"])
        df_test["LWP_pred"] = y_pred

        y_pred_proba = clf.predict_proba(X_test)[:, 1] 

        new = pd.DataFrame(columns=df_results.columns, data=[[main_country, window_size, recall_score(y_test,y_pred), precision_score(y_test,y_pred), accuracy_score(y_test,y_pred), roc_auc_score(y_test.values, y_pred_proba)]])
        df_results = pd.concat([df_results, new], axis=0)


        print("""       Recall:""",recall_score(y_test,y_pred))
        print("""       Precision:""",precision_score(y_test,y_pred))
        print("""       Accuracy:""",accuracy_score(y_test,y_pred))
        print("""       Roc auc:""",roc_auc_score(y_test.values, y_pred_proba))





country:  AT
   window_size = 1
       Recall: 0.701732150401352
       Precision: 0.656781336496639
       Accuracy: 0.6563318777292576
       Roc auc: 0.7069074300934429
   window_size = 10
       Recall: 0.699614890885751
       Precision: 0.664904432696218
       Accuracy: 0.6665938387590125
       Roc auc: 0.7157938672901767
   window_size = 20
       Recall: 0.6903445342407486
       Precision: 0.6679012345679012
       Accuracy: 0.6644075207695671
       Roc auc: 0.7137845650236794
country:  BE
   window_size = 1
       Recall: 0.6971867007672634
       Precision: 0.6496663489037178
       Accuracy: 0.7102620087336244
       Roc auc: 0.774029570088905
   window_size = 10
       Recall: 0.71850699844479
       Precision: 0.6550094517958412
       Accuracy: 0.7218702206685602
       Roc auc: 0.7924053130858466
   window_size = 20
       Recall: 0.6886409736308317
       Precision: 0.6679783571077226
       Accuracy: 0.7181897682553564
       Roc auc: 0.7823738721813304
country:  B